# Processing scripts for generating figures for Dafne paper

## Processing log file

### Anonymization

In [8]:
import os
from typing import Iterable
import re

DELETE_NONANON_FILES = False

def find_elements_pattern(string_list: Iterable[str], pattern: str):
    return [ s for s in string_list if re.match(pattern, s) is not None]

def anonymize_line(line: str):
    pattern = 'accessed by \S+'
    rep = 'accessed by XXXXXXX'
    line = re.sub(pattern, rep, line)
    pattern2 = 'Log message from \S+'
    rep2 = 'Log message from XXXXXXX'
    line = re.sub(pattern2, rep2, line)
    pattern3 = r'([0-9]+)_\S+.model'
    rep3 = r'\1_XXXXXXX.model'
    line = re.sub(pattern3, rep3, line)
    return line
    

def anonymize_file(filename: str):
    out_filename = filename[:-3] + 'anon.txt'
    with open(filename, 'r') as input_file:
        with open(out_filename, 'w') as output_file:
            for line in input_file:
                output_file.write(anonymize_line(line))

files = os.listdir()
nonanon_log_files = find_elements_pattern(files, 'log.*(?<!anon)\.txt')
for file in nonanon_log_files:
    anonymize_file(file)
    if DELETE_NONANON_FILES:
        os.unlink(file)
    print('Anonymized', file)


Anonymized log_20220831_orig.txt


### Read in anonymized file

In [18]:
from dateutil.parser import parse
import pandas as pd

files = os.listdir()
anon_log_files = find_elements_pattern(files, 'log.*anon\.txt')
anon_log_file = anon_log_files[0]
print('Reading from', anon_log_file)

def split_log_line(line):
    date_part = line[:27]
    text_part = line[27:].strip()
    date = parse(date_part)
    return (date, text_part)

with open(anon_log_file,'r') as f:
    log_lines = [ split_log_line(line) for line in f ]
    
log_frame = pd.DataFrame(log_lines, columns = ['DateTime', 'Message'])

Reading from log_20220831_orig.anon.txt


### Filter dates and messages

In [17]:
from datetime import datetime

date_start = 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3688 entries, 0 to 3687
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   DateTime  3688 non-null   datetime64[ns]
 1   Message   3688 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 57.8+ KB
